In [3]:
import numpy as np
import pandas as pd
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#systeme
import sys
sys.setrecursionlimit(200000)

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams

#import warnings; warnings.simplefilter('ignore')

import re
import string

#import spacy
#from spacy_lefff import lefffLemmatizer

#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from nltk.stem.snowball import FrenchStemmer

import fr_core_news_sm
nlp = fr_core_news_sm.load()

#Visualisation
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
#Parametres generaux d'affichage des graphiques
plt.rcParams['figure.figsize'] = (25, 8)
plt.rcParams['figure.titlesize'] = 20
plt.rcParams['font.size'] = 15


#%load_ext Cython

#wolf
#from frenetic.packet import *
#fwn = FreNetic("C:/Users/Utilisateur/Desktop/simplon/projetA1/wolf.xml") 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


C:\Users\Dell\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


<h1 align='center'> Traitement du langage naturel </h1>
<h2 align='center'> Analyse lexicale de sentiment de tweets Français</h2>
<br>
<br>
<hr>
<h2 align='center'><strong>1ere Partie: </strong>Pre-processing</h2>


### 1/ Import du csv

In [4]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [5]:
tweets = pd.read_csv("./french-tweets-politique/tweets_unique.csv", encoding='utf-8')

In [6]:
tweets.head()

,Unnamed: 0,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [7]:
tweets.rename(columns={'Unnamed: 0':'num'}, inplace=True)

In [8]:
tweets.head()

,num,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


In [9]:
tw = tweets.copy()

### 2/ Normalisation du texte

In [10]:
low = lambda x: str(x).lower()
tweets['text'] = tweets.text.apply(low)
tweets.head()

,num,date,text,hashtags
0,0,Sun Mar 18 13:45:13 +0000 2018,au lieu d'expulser des diplomates russes il eu...,NaN
1,1,Sun Mar 18 15:47:19 +0000 2018,"selon l’ex ambassadeur anglais en ouzbékistan,...",NaN
2,2,Sun Mar 18 11:48:32 +0000 2018,@philipj37 @eugeniebastie tellement vrai.\r\ns...,NaN
3,3,Sun Mar 18 15:41:24 +0000 2018,"bombe: le capitaine barril, ancien commandant ...",NaN
4,4,Sun Mar 18 14:32:39 +0000 2018,dans la même semaine #macron nomme le très amb...,"['Macron', 'Bellatar', 'ConseilDesVilles']"


### 3/ Supression des caracteres speciaux et double espaces

In [11]:
#tweets['text'] = tweets['text'].apply(lambda x: re.sub('[!@#$:’).";,?&%=>(<''©-]', '', x.lower()))
tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text'].head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon l’ex ambassadeur anglais en ouzbékistan,...
2    @philipj37 @eugeniebastie tellement vrai.\r\ns...
3    bombe: le capitaine barril, ancien commandant ...
4    dans la même semaine #macron nomme le très amb...
Name: text, dtype: object

In [12]:
for i in range(len(tweets['text'])):
    tweets['text'][i] = " ".join([word for word in tweets['text'][i].split()
                                if 'http' not in word and '@' not in word and '<' not in word])
tweets['text'].head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon l’ex ambassadeur anglais en ouzbékistan,...
2    tellement vrai. s'occuper de trump et de pouti...
3    bombe: le capitaine barril, ancien commandant ...
4    dans la même semaine #macron nomme le très amb...
Name: text, dtype: object

### 4/ Tokénisation

In [13]:
tweets.dtypes

num         object
date        object
text        object
hashtags    object
dtype: object

In [14]:
tweets['text'].astype('str', inplace=True).head()

0    au lieu d'expulser des diplomates russes il eu...
1    selon l’ex ambassadeur anglais en ouzbékistan,...
2    tellement vrai. s'occuper de trump et de pouti...
3    bombe: le capitaine barril, ancien commandant ...
4    dans la même semaine #macron nomme le très amb...
Name: text, dtype: object

In [15]:
clean = lambda tx: word_tokenize(tx, language='french')
tweets.text = tweets.text.apply(clean)
tweets.text.head()

0    [au, lieu, d'expulser, des, diplomates, russes...
1    [selon, l, ’, ex, ambassadeur, anglais, en, ou...
2    [tellement, vrai, ., s'occuper, de, trump, et,...
3    [bombe, :, le, capitaine, barril, ,, ancien, c...
4    [dans, la, même, semaine, #, macron, nomme, le...
Name: text, dtype: object

### 5/ Stop-Words

In [16]:
stopW = stopwords.words('french')
punc = string.punctuation
#print(stopW)
filtered_sentence = [w for w in tweets['text'] if not w in stopW] 
# remove URLs, RTs, and twitter handles
# for i in range(len(filtered_sentence)):
#     filtered_sentence[i] =" ".join([word for word in filtered_sentence[i]
#                                if 'http' not in word  and '<' not in word])
filtered_sentence[0]
#stopW = stopW.extend(punc)
stopW.extend(punc)

In [17]:
def filtration(tk: list ):#texte tokenisé sous format de liste
    l_filtree = []
    for word in tk:
        word = word.strip()
        if word not in stopW:
            l_filtree.append(str(word))
    return l_filtree

In [18]:
filtered_sentence = tweets['text'].apply(filtration)
filtered_sentence.head()

0    [lieu, d'expulser, diplomates, russes, plus, c...
1    [selon, ’, ex, ambassadeur, anglais, ouzbékist...
2    [tellement, vrai, s'occuper, trump, poutine, c...
3    [bombe, capitaine, barril, ancien, commandant,...
4    [semaine, macron, nomme, très, ambigu, l'islam...
Name: text, dtype: object


### 6/ Racinalisation

In [19]:
#Lemmatization
stemmer = FrenchStemmer()
stem = lambda x :[stemmer.stem(w) for w in x]
tstem = filtered_sentence.apply(stem)

In [20]:
tstem.head()

0    [lieu, d'expuls, diplomat, russ, plus, concret...
1    [selon, ’, ex, ambassadeur, anglais, ouzbékist...
2    [tel, vrai, s'occup, trump, poutin, c'est, tel...
3    [bomb, capitain, barril, ancien, command, gign...
4    [semain, macron, nomm, tres, ambigu, l'islam, ...
Name: text, dtype: object

In [21]:
tstem = tstem.apply(filtration)
tstem.head()

0    [lieu, d'expuls, diplomat, russ, plus, concret...
1    [selon, ’, ex, ambassadeur, anglais, ouzbékist...
2    [tel, vrai, s'occup, trump, poutin, c'est, tel...
3    [bomb, capitain, barril, ancien, command, gign...
4    [semain, macron, nomm, tres, ambigu, l'islam, ...
Name: text, dtype: object

<br>
<br>
<hr>
<h2 align='center'><b>2eme Partie: </b>Attribution d'un taux de polarite de sentiment</h2>

### 1/ Agregation des donnees

In [22]:
# Import du fichier Feel.csv
fe = pd.read_csv('./nlp_fr/FEEL.csv', sep=';', encoding='utf-8')
fe.set_index('word', inplace=True)
fe.head(20)
#fe.loc['?il']

,id,polarity,joy,fear,sadness,anger,surprise,disgust
word,,,,,,,,
à ce endroit là,1,positive,0,0,0,0,0,0
à le hâte,2,negative,0,1,0,0,1,0
à part,3,negative,0,0,1,0,0,0
à pic,4,negative,0,1,0,0,0,0
à rallonge,5,negative,0,0,1,0,0,0
abasourdir,6,negative,0,0,0,0,1,0
ablation,7,negative,0,1,0,0,0,1
abominable,8,negative,0,1,0,0,0,1
abrupt,9,negative,0,1,0,0,0,0


In [23]:
#import motsJoyeux et motNegatifs
with open('motsJoyeux', 'r', encoding='utf-8-sig', ) as f:
    x = f.read().split(',')
motsJoyeux = [mot.lower().strip() for mot in x]
p = pd.Series('positif', index=motsJoyeux)
p = p.to_frame().rename(columns={0:'polarity'})


with open('motsNegatifs', 'r', encoding='utf-8-sig') as f:
    x = f.read().split(',')
motsNegatifs = [mot.lower().strip() for mot in x]
n = pd.Series('negatif', index=motsNegatifs)
n = n.to_frame().rename(columns={0:'polarity'})

p.head()

,polarity
à bas prix,positif
à couper le souffle,positif
à croissance rapide,positif
à être vu,positif
à juste titre,positif


In [24]:
# Aggregation des donnees
f = fe.copy()
print(len(f))
f = pd.concat([f,p], axis=0, sort=False)
print(len(f))
f = pd.concat([f,n], axis=0, sort=False)
print(len(f))
print(f.index.value_counts().head()) 
f.head()

14127
15627
15783
étourdi       3
extase        3
aventureux    3
sagacité      3
bien          3
dtype: int64


,id,polarity,joy,fear,sadness,anger,surprise,disgust
à ce endroit là,1.0,positive,0.0,0.0,0.0,0.0,0.0,0.0
à le hâte,2.0,negative,0.0,1.0,0.0,0.0,1.0,0.0
à part,3.0,negative,0.0,0.0,1.0,0.0,0.0,0.0
à pic,4.0,negative,0.0,1.0,0.0,0.0,0.0,0.0
à rallonge,5.0,negative,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
print(len(f.groupby(f.index).first())== len(f[~f.index.duplicated(keep='first')]))

True


In [26]:
f = f[~f.index.duplicated(keep='first')]
len(f)
f.head()

,id,polarity,joy,fear,sadness,anger,surprise,disgust
à ce endroit là,1.0,positive,0.0,0.0,0.0,0.0,0.0,0.0
à le hâte,2.0,negative,0.0,1.0,0.0,0.0,1.0,0.0
à part,3.0,negative,0.0,0.0,1.0,0.0,0.0,0.0
à pic,4.0,negative,0.0,1.0,0.0,0.0,0.0,0.0
à rallonge,5.0,negative,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
print(f.index.value_counts().head()) 
f.head()

qui monter en flêche    1
fauvette                1
révolu                  1
exubérance              1
fardeau                 1
dtype: int64


,id,polarity,joy,fear,sadness,anger,surprise,disgust
à ce endroit là,1.0,positive,0.0,0.0,0.0,0.0,0.0,0.0
à le hâte,2.0,negative,0.0,1.0,0.0,0.0,1.0,0.0
à part,3.0,negative,0.0,0.0,1.0,0.0,0.0,0.0
à pic,4.0,negative,0.0,1.0,0.0,0.0,0.0,0.0
à rallonge,5.0,negative,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
import xml.etree.ElementTree as ET

#do this if running in jupyter
pd.set_option('display.max_columns', None)

# convert XML to dataframe (special WOLF xml format)
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for att in child.attrib:
            record[att] = child.attrib[att]
        all_records.append(record)
    df = pd.DataFrame(all_records)
    return df

In [29]:
with open('nlp_fr/fr-sentiment.xml', 'r',encoding='utf-8') as a:
    xml_data = a.read()
df = xml2df(xml_data)

In [30]:
df.head()

,confidence,form,intensity,polarity,pos,subjectivity
0,0.9,abandonné,1.0,-0.30,JJ,0.40
1,0.8,abandonnée,1.0,-0.30,JJ,0.40
2,0.8,abandonnées,1.0,-0.30,JJ,0.40
3,0.8,abandonnés,1.0,-0.30,JJ,0.40
4,0.7,abasourdi,1.0,0.24,JJ,0.55


In [31]:
# Racinaliser tous le mots des deux df pour pouvoir les assembler
from nltk.stem.snowball import FrenchStemmer
fepp = f.copy()
sent = df.copy()

#Creation du stemmer
stemmer = FrenchStemmer()

#Stemmatization
fepp = fepp.reset_index()
fepp.rename(columns={'index':'word'},inplace=True)
fepp.word= fepp.word.apply(stemmer.stem)
sent.form = sent.form.apply(stemmer.stem)
sent = sent.set_index('form')
fepp =fepp.set_index('word')

# Supprime les doubblons
fepp = fepp[~fepp.index.duplicated(keep='first')]
sent = sent[~sent.index.duplicated(keep='first')]
fepp.head()

,id,polarity,joy,fear,sadness,anger,surprise,disgust
word,,,,,,,,
à ce endroit là,1.0,positive,0.0,0.0,0.0,0.0,0.0,0.0
à le hât,2.0,negative,0.0,1.0,0.0,0.0,1.0,0.0
à part,3.0,negative,0.0,0.0,1.0,0.0,0.0,0.0
à pic,4.0,negative,0.0,1.0,0.0,0.0,0.0,0.0
à rallong,5.0,negative,0.0,0.0,1.0,0.0,0.0,0.0


In [32]:
# Concatenation des dataframes
test = fepp.copy()
test =test.merge(sent, how='inner', left_index=True, right_index=True)
test.polarity_y = test.polarity_y.astype('Float64')
test.subjectivity = test.subjectivity.astype('Float64')
test.head()

,id,polarity_x,joy,fear,sadness,anger,surprise,disgust,confidence,intensity,polarity_y,pos,subjectivity
abasourd,6.0,negative,0.0,0.0,0.0,0.0,1.0,0.0,0.7,1.0,0.24,JJ,0.55
abomin,8.0,negative,0.0,1.0,0.0,0.0,0.0,1.0,0.9,1.0,-0.90,JJ,1.00
absent,10.0,negative,0.0,1.0,1.0,0.0,0.0,0.0,0.9,1.0,-0.10,JJ,0.00
absorb,11.0,positive,0.0,0.0,0.0,0.0,1.0,0.0,0.7,1.0,0.30,JJ,0.60
absurd,12.0,negative,0.0,0.0,0.0,1.0,1.0,1.0,0.9,1.0,0.40,JJ,0.70


### 2/ Fonction de calcul des polarite

In [33]:
# Calculpolarite multiply the indices of polarity and subjectivity of words found in the list and returns the sum.
# A positive result means a positive feeling while a negative result means a feeling negative.
def calculpolarite(text:list):
    s=sum([test.loc[word].polarity_y*test.loc[word].subjectivity for word in text if word in test.index.values])
    return s

In [34]:
polarite = tstem.apply(calculpolarite)

In [35]:
polarite.head()

0    0.4580
1    0.3200
2    0.0350
3    0.0000
4    0.0365
Name: text, dtype: float64

In [36]:
tweets['sentiment'] = polarite
tweets.head()

,num,date,text,hashtags,sentiment
0,0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,0.4580
1,1,Sun Mar 18 15:47:19 +0000 2018,"[selon, l, ’, ex, ambassadeur, anglais, en, ou...",NaN,0.3200
2,2,Sun Mar 18 11:48:32 +0000 2018,"[tellement, vrai, ., s'occuper, de, trump, et,...",NaN,0.0350
3,3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, :, le, capitaine, barril, ,, ancien, c...",NaN,0.0000
4,4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, #, macron, nomme, le...","['Macron', 'Bellatar', 'ConseilDesVilles']",0.0365


## Preparation des donnees pour l'export

In [37]:
tweets.hashtags.replace("[][']", '', regex=True, inplace=True)

In [38]:
textjoin = lambda l : ','.join([w.strip() for w in l])
t = tstem.apply(textjoin)
tweets['words'] = t

In [39]:
tweets.head()

,num,date,text,hashtags,sentiment,words
0,0,Sun Mar 18 13:45:13 +0000 2018,"[au, lieu, d'expulser, des, diplomates, russes...",NaN,0.4580,"lieu,d'expuls,diplomat,russ,plus,concret,plus,..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"[selon, l, ’, ex, ambassadeur, anglais, en, ou...",NaN,0.3200,"selon,’,ex,ambassadeur,anglais,ouzbékistan,``,..."
2,2,Sun Mar 18 11:48:32 +0000 2018,"[tellement, vrai, ., s'occuper, de, trump, et,...",NaN,0.0350,"tel,vrai,s'occup,trump,poutin,c'est,tel,plus,l..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"[bombe, :, le, capitaine, barril, ,, ancien, c...",NaN,0.0000,"bomb,capitain,barril,ancien,command,gign,«,’,p..."
4,4,Sun Mar 18 14:32:39 +0000 2018,"[dans, la, même, semaine, #, macron, nomme, le...","Macron, Bellatar, ConseilDesVilles",0.0365,"semain,macron,nomm,tres,ambigu,l'islam,bellata..."


In [40]:
#Recuperer la colonne text du df de depart
tweets['text'] = tw.text
tweets.head()

,num,date,text,hashtags,sentiment,words
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NaN,0.4580,"lieu,d'expuls,diplomat,russ,plus,concret,plus,..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NaN,0.3200,"selon,’,ex,ambassadeur,anglais,ouzbékistan,``,..."
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NaN,0.0350,"tel,vrai,s'occup,trump,poutin,c'est,tel,plus,l..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NaN,0.0000,"bomb,capitain,barril,ancien,command,gign,«,’,p..."
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",0.0365,"semain,macron,nomm,tres,ambigu,l'islam,bellata..."


In [41]:
#Remplace les nan par NO HASHTAG
tweets.hashtags.replace(np.nan, 'NO HASHTAG' ,inplace=True)
tweets.head()

,num,date,text,hashtags,sentiment,words
0,0,Sun Mar 18 13:45:13 +0000 2018,Au lieu d'expulser des diplomates Russes il eu...,NO HASHTAG,0.4580,"lieu,d'expuls,diplomat,russ,plus,concret,plus,..."
1,1,Sun Mar 18 15:47:19 +0000 2018,"Selon l’ex Ambassadeur anglais en Ouzbékistan,...",NO HASHTAG,0.3200,"selon,’,ex,ambassadeur,anglais,ouzbékistan,``,..."
2,2,Sun Mar 18 11:48:32 +0000 2018,@PhilipJ37 @EugenieBastie Tellement vrai.\r\nS...,NO HASHTAG,0.0350,"tel,vrai,s'occup,trump,poutin,c'est,tel,plus,l..."
3,3,Sun Mar 18 15:41:24 +0000 2018,"BOMBE: Le capitaine Barril, ancien commandant ...",NO HASHTAG,0.0000,"bomb,capitain,barril,ancien,command,gign,«,’,p..."
4,4,Sun Mar 18 14:32:39 +0000 2018,Dans la même semaine #Macron nomme le très amb...,"Macron, Bellatar, ConseilDesVilles",0.0365,"semain,macron,nomm,tres,ambigu,l'islam,bellata..."


In [42]:
# Export du df en CSV
tweets.to_csv("tweetsAnalyse.csv", index=False, encoding='utf-8')

## Fonction de determination de polarite a partir d'un texte

In [53]:
# The function polarity_feeling takes a string as entry
# Returns the text's feeling polarity's rate in a type np.float 
def polarity_feeling(text: str):
    # text to lower case 
    # delete punctuation
    #text =re.sub('[!@#$:’).";,?&%=>(<''©-]', '', str(text).lower())
    # delete doubble whitespace
    text = re.sub('  ', ' ', text)
    # delete @, > and 'http'
    text = " ".join([word for word in text.split()
                                if 'http' not in word and '@' not in word and '<' not in word])
    # Tokenisation
    text = word_tokenize(text)
    text = filtration(text)

    # Lemmatization
    text = stem(text)
    # delete stopwords
    text = filtration(text)
    # define the feeling polarity
    return round(calculpolarite(text),2)
    
    

In [55]:
polarity_feeling('Vive l\'administration française. Elle est belle!')

0.32

 <img align='center' src="wordcloud2.PNG">